In [27]:
import RIM
import torch
from torch import nn
import numpy as np
import pandas as pd
import os
import RIM_SequentialDataloader
from RIM import RIM
import faiss
import csv
from tqdm import tqdm

In [78]:
r_path = "L:/5104"
pool_label = np.load(os.path.join(r_path, 'pool_label.npy'))
# test_label = np.load(os.path.join(r_path, 'test_label.npy'))
test_label = np.load(os.path.join(r_path, 'sample_test_label.npy'))

In [25]:
def RetrieveKNN(xq, k):
    index = faiss.read_index(os.path.join(r_path, 'faiss_index_1.index'))
    D1, I1 = index.search(xq, k)
    index = faiss.read_index(os.path.join(r_path, 'faiss_index_4.index'))
    D2, I2 = index.search(xq, k)
    index = faiss.read_index(os.path.join(r_path, 'faiss_index_2.index'))
    D3, I3 = index.search(xq, k)
    index = faiss.read_index(os.path.join(r_path, 'faiss_index_3.index'))
    D4, I4 = index.search(xq, k)
    I2 = I2 + 20000000
    I3 = I3 + 39000000
    I4 = I4 + 60000000
    I = np.concatenate((I1, I2, I3, I4), axis=1)
    D = np.concatenate((D1, D2, D3, D4), axis=1)
    ind_sort = D.argsort(axis=1)
    D_sort = np.take_along_axis(D, ind_sort, axis=1)
    I_sort = np.take_along_axis(I, ind_sort, axis=1)
    return I_sort, D_sort

In [74]:
batch_size = 1000
csv_file = os.path.join(r_path, 'test_embed.csv')
for chunk in pd.read_csv(csv_file, chunksize=batch_size):
    xq = chunk.values
    I, D = RetrieveKNN(xq, 10)
    break

In [79]:
def Predict(I, k):
    Preds = np.zeros(I.shape[0])
    KNN_label = pool_label[I[:, :k]]
    tmp = np.sum(KNN_label, axis=1)
    Preds[tmp / k >= 0.5] = 1
    return Preds

In [80]:
print(f'Accuracy: {np.sum(Predict(I, 10) == test_label[:1000]) / 1000}')

Accuracy: 0.941
